In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import scipy as sp
import string
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

import nltk
# import scikitplot as skplt
import re, string
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
STOPWORDS = set(stopwords.words('russian'))
#list of special characters.You can use regular expressions too

# importing the PorterStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize,word_tokenize
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
#importing the CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import SnowballStemmer
# from pymorphy2 import MorphAnalyzer

# start text processing with vectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [2]:
ps = SnowballStemmer(language="russian")

In [3]:
data_train = pd.read_json('train.json')
data_train

,text,id,sentiment
0,Досудебное расследование по факту покупки ЕНПФ...,1945,negative
1,Медики рассказали о состоянии пострадавшего му...,1957,negative
2,"Прошел почти год, как железнодорожным оператор...",1969,negative
3,По итогам 12 месяцев 2016 года на территории р...,1973,negative
4,Астана. 21 ноября. Kazakhstan Today - Агентств...,1975,negative
...,...,...,...
8258,"Как мы писали еще весной, для увеличения сбыта...",10312,positive
8259,Но молодой министр национальной экономики Биши...,10313,negative
8260,\n \nВ ЕНПФ назначен новый председатель правле...,10314,neutral
8261,В Алматы у отделения банка произошло нападение...,10315,negative


In [4]:
data_test = pd.read_json('test.json')
data_test

,text,id
0,"Как сообщает пресс-служба акимата Алматы, для ...",0
1,Казахстанские авиакомпании перевозят 250 тысяч...,1
2,На состоявшемся под председательством Касым-Жо...,2
3,В ОАЭ состоялись переговоры между казахстанско...,3
4,12 вагонов грузового поезда сошли с путей в Во...,4
...,...,...
2051,На официальной странице общественного движения...,2079
2052,"официальный курс – 330,55 тенге за Доллар США ...",2083
2053,"«Базовая ставка, которая сейчас составляет 12%...",2084
2054,На начальном этапе за неоплату парковки на при...,2087


In [5]:
data_train['sentiment'] = data_train['sentiment'].map(lambda x: 0 if x == 'negative' else 1 if x == 'positive' else 2)

In [6]:
data_train

,text,id,sentiment
0,Досудебное расследование по факту покупки ЕНПФ...,1945,0
1,Медики рассказали о состоянии пострадавшего му...,1957,0
2,"Прошел почти год, как железнодорожным оператор...",1969,0
3,По итогам 12 месяцев 2016 года на территории р...,1973,0
4,Астана. 21 ноября. Kazakhstan Today - Агентств...,1975,0
...,...,...,...
8258,"Как мы писали еще весной, для увеличения сбыта...",10312,1
8259,Но молодой министр национальной экономики Биши...,10313,0
8260,\n \nВ ЕНПФ назначен новый председатель правле...,10314,2
8261,В Алматы у отделения банка произошло нападение...,10315,0


In [7]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]+|[\d]+', r'',text).strip()
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    text = regex.sub(' ', text)
#     text = re.sub(r'[^0-9a-zA-Z]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = " ".join(word for word in text.split() if word not in STOPWORDS)
    text = " ".join(ps.stem(word) for word in text.split())
    return text

data_train['clean_text'] = data_train['text'].apply(clean_text)
data_train.head()

,text,id,sentiment,clean_text
0,Досудебное расследование по факту покупки ЕНПФ...,1945,0,досудебн расследован факт покупк енпф пакет об...
1,Медики рассказали о состоянии пострадавшего му...,1957,0,медик рассказа состоян пострада мужчин котор с...
2,"Прошел почти год, как железнодорожным оператор...",1969,0,прошел год железнодорожн оператор запрет экспл...
3,По итогам 12 месяцев 2016 года на территории р...,1973,0,итог месяц год территор республик выпущ единиц...
4,Астана. 21 ноября. Kazakhstan Today - Агентств...,1975,0,аста ноябр kazakhstan today агентств рк дел го...


In [8]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.3 MB/s eta 0:00:00


In [9]:
import torch
import random
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import f1_score

In [10]:
df_bert = data_train[['clean_text', 'sentiment']].copy()

In [11]:
# Splits data into features and target
X = df_bert['clean_text']
y = df_bert['sentiment']

# Splits data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
device_id = 0
device = 'cpu' if device_id == -1 else f'cuda:{device_id}' # 'cuda:0' id GPU

In [13]:
device

'cuda:0'

In [14]:
# X_train.values

In [15]:
# Loads the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [16]:
encoded_data_train = tokenizer.batch_encode_plus(X_train.values,
                                                add_special_tokens=True,
                                                return_attention_mask=True,
                                                pad_to_max_length=True,
                                                max_length=256,
                                                 return_tensors='pt'
                                                )
encoded_data_val = tokenizer.batch_encode_plus(X_test.values,
                                                add_special_tokens=True,
                                                return_attention_mask=True,
                                                pad_to_max_length=True,
                                                max_length=256,
                                                 return_tensors='pt'
                                                )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [17]:
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(y_test.values)

In [18]:
train_inputs = torch.tensor(encoded_data_train['input_ids']).to(device)
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(y_train.values)

In [19]:
dataset_train = TensorDataset(train_inputs, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [20]:
from transformers import AdamW
# from transformers.optimization import get_linear_scheduler_with_warmup

In [21]:
#SETTING MODEL
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3, output_attentions=False, output_hidden_states=False )

#CREATING DATA LOADERS
dataloader_train = DataLoader(dataset_train,sampler = RandomSampler(dataset_train), batch_size= 32)


dataloader_val = DataLoader(dataset_val, sampler = RandomSampler(dataset_val), batch_size= 32)


#SETTING OPTIMIZERS

op = AdamW(model.parameters(),lr=1e-5,eps=1e-8)

epochs = 30

# scheduler = get_linear_schedule_with_warmup(op, num_warmup_steps=0, num_training_steps=len(dataloader_train)*epochs)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
#FUNCTION TO CALCULATE F1 SCORE
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds,axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

#FUNCTION FOR CALCULATING ACCURACY PER CLASS
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v:k for k,v in label_dict.items()}

    preds_flat = np.argmax(preds,axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(label_dict_inverse[label])
        print("accuracy ", len(y_preds[y_preds==label])/len(y_true))

#FUNCTION FOR MODEL EVALUATION
def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

In [23]:
 model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [24]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()

    loss_train_total = 0

    progress_bar = tqdm(dataloader_train,
                        desc ='Epoch {:1d}'.format(epoch),
                        leave=False,
                       disable=False
                       )
    for batch in progress_bar:
        model.zero_grad()

        batch = tuple(b.to(device) for b in batch)

        inputs = { 'input_ids' : batch[0],
                 'attention_mask' : batch[1],
                 'labels' : batch[2]
                 }
        outputs = model(**inputs)


        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        op.step()
        # scheduler.step()

        progress_bar.set_postfix({'training_loss':'{:.3f}'.format(loss.item()/len(batch))})

    # THIS SECTION OF CODE IS JUST FOR PRINTING VALUES AFTER EACH EPOCH.
    torch.save(model.state_dict(), f'BERT_ft_epoch{epoch}.model')
    tqdm.write(f'\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 score (weighted): {val_f1}')

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/207 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.0211019455522732
Validation loss: 1.0156272363204222
F1 score (weighted): 0.42764997572078195


Epoch 2:   0%|          | 0/207 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.9752339340638423
Validation loss: 0.9443304664813555
F1 score (weighted): 0.477407468861397


...


Epoch 28
Training loss: 0.20675329734449802
Validation loss: 1.4019250159080212
F1 score (weighted): 0.6349804495830872


Epoch 29:   0%|          | 0/207 [00:00<?, ?it/s]


Epoch 29
Training loss: 0.19896084664992375
Validation loss: 1.491348657470483
F1 score (weighted): 0.621965414652096


Epoch 30:   0%|          | 0/207 [00:00<?, ?it/s]


Epoch 30
Training loss: 0.19516758138430867
Validation loss: 1.4812466502189636
F1 score (weighted): 0.6252592324210974


In [25]:
df = pd.read_json('train.json')
df

,text,id,sentiment
0,Досудебное расследование по факту покупки ЕНПФ...,1945,negative
1,Медики рассказали о состоянии пострадавшего му...,1957,negative
2,"Прошел почти год, как железнодорожным оператор...",1969,negative
3,По итогам 12 месяцев 2016 года на территории р...,1973,negative
4,Астана. 21 ноября. Kazakhstan Today - Агентств...,1975,negative
...,...,...,...
8258,"Как мы писали еще весной, для увеличения сбыта...",10312,positive
8259,Но молодой министр национальной экономики Биши...,10313,negative
8260,\n \nВ ЕНПФ назначен новый председатель правле...,10314,neutral
8261,В Алматы у отделения банка произошло нападение...,10315,negative


In [26]:
possible_labels = df.sentiment.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

df['label'] = df.sentiment.replace(label_dict)
df.head()

,text,id,sentiment,label
0,Досудебное расследование по факту покупки ЕНПФ...,1945,negative,0
1,Медики рассказали о состоянии пострадавшего му...,1957,negative,0
2,"Прошел почти год, как железнодорожным оператор...",1969,negative,0
3,По итогам 12 месяцев 2016 года на территории р...,1973,negative,0
4,Астана. 21 ноября. Kazakhstan Today - Агентств...,1975,negative,0


In [27]:
_, predictions, true_val = evaluate(dataloader_val) #why _ ? reason behind this is evaluate function return 3 values and i don't require the 1st value i.e., loss_val_avg

accuracy_per_class(predictions, true_val)

negative
accuracy  0.5333333333333333
positive
accuracy  0.6654275092936803
neutral
accuracy  0.6313253012048192


In [65]:
model.eval()

batch = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(data_test['text'][0]))
with torch.no_grad():
            predict = model(torch.tensor(batch).unsqueeze(0).to(device)) # 1 x T x N_classes
            predict
            print(predict)
            print(type(predict))
            for i in predict:
                print(type(i))
            # preds = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
            # print(preds)

SequenceClassifierOutput(loss=None, logits=tensor([[-2.6063, -1.1637,  3.6655]], device='cuda:0'), hidden_states=None, attentions=None)
<class 'transformers.modeling_outputs.SequenceClassifierOutput'>
<class 'str'>


In [46]:
def predict(data_test):
    ress = []
    model.eval()

    for batch in tqdm(data_test['text']):
        res = {}
        res['text'] = batch
        batch = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(batch))
        with torch.no_grad():
            predict = model(torch.tensor(batch).unsqueeze(0).to(device)) # 1 x T x N_classes

            print(predict)
            preds = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()

        if preds == 1:
                      label = 'positive'
        elif preds == 0:
                      label = 'negative'
        else:
                      label = 'neutral'
        res['sentiment'] = label
        ress.append(res)

        return pd.DataFrame.from_dict(ress)

In [68]:
def predict(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        return_attention_mask=True,
        pad_to_max_length=True,
        max_length=256,
        return_tensors='pt'
    )

    out = {
          'text': text,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten()
      }

    input_ids = out["input_ids"].to(device)
    attention_mask = out["attention_mask"].to(device)
    model.eval()
    with torch.no_grad():
            outputs = model(
                input_ids=input_ids.unsqueeze(0),
                attention_mask=attention_mask.unsqueeze(0)
            )

    prediction = torch.argmax(outputs.logits, dim=1).cpu().numpy()[0]

    return prediction

In [69]:
predict(data_test['text'][0])

2

In [70]:
ress = []
for sentence in tqdm(data_test['text']):
    # print(sentence)
    res = {}
    res['text'] = sentence
    # print(res['text'])
    preds = predict(sentence)
    # print(preds)
    if preds == 1:
                  label = 'positive'
    elif preds == 0:
                  label = 'negative'
    else:
                  label = 'neutral'
    res['sentiment'] = label
    ress.append(res)

  0%|          | 0/2056 [00:00<?, ?it/s]

In [71]:
itog = pd.DataFrame.from_dict(ress)
itog

,text,sentiment
0,"Как сообщает пресс-служба акимата Алматы, для ...",neutral
1,Казахстанские авиакомпании перевозят 250 тысяч...,positive
2,На состоявшемся под председательством Касым-Жо...,neutral
3,В ОАЭ состоялись переговоры между казахстанско...,neutral
4,12 вагонов грузового поезда сошли с путей в Во...,neutral
...,...,...
2051,На официальной странице общественного движения...,neutral
2052,"официальный курс – 330,55 тенге за Доллар США ...",neutral
2053,"«Базовая ставка, которая сейчас составляет 12%...",positive
2054,На начальном этапе за неоплату парковки на при...,neutral


In [72]:
itog.value_counts("sentiment")

sentiment
neutral     1118
positive     810
negative     128
dtype: int64